In [49]:
#DNS TUNNELING PCAP FILE FOR EXAMPLE 

#ioline = 'C://Users//igor.beryozkin//Downloads/dns-tunnel-iodine.pcap'
#a = rdpcap(ioline)

In [136]:
import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.types import *

from matplotlib import pyplot as plt
from plotly import graph_objects as go

from scapy.all import *
import sys
import os

import math

In [51]:
spark = SparkSession.builder.appName('main').getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
sparkContext = spark.sparkContext
sparkContext, spark

(<SparkContext master=local[*] appName=main>,
 <pyspark.sql.session.SparkSession at 0x1ae18d97eb0>)

In [149]:
main_folder = "C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP"

files_list = []
for file in os.listdir(main_folder):
    pcap_folder = 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\{0}'.format(file)
    files_list.append(pcap_folder)
    
    


In [156]:
# Files weight
def GetSize(files):
    size = []
    
    for file in os.listdir(files):
        
        pcap_folder = 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\{0}'.format(file)
    
        
        size.append(os.stat(pcap_folder).st_size)
        
    return print((sum(size)) // 1073741824, 'GB') 

GetSize(main_folder)

5 GB


In [53]:
#Таблица признаков и их описание
F_columns = ['№', 'Признак', 'Описание']
F_data = [(1, 'Размер запроса в байтах', 'Отклонение от средних показателей 40-60 байт в сторону увеличения может говорить о скрытом канале'),
          (2, 'Необычный тип трафика', 'Если в трафике используются данные вида TXT, NULL, KEY - это говорит об аномалии'),
          (3, 'Длина имени запроса', 'Имена используемые злоумышленниками могут достигать 200 символов - средняя 30 символов')]

In [54]:
feauture_DF = spark.createDataFrame(F_data, F_columns)

In [55]:
feauture_DF.show(truncate= 0)

+---+-----------------------+-------------------------------------------------------------------------------------------------+
|№  |Признак                |Описание                                                                                         |
+---+-----------------------+-------------------------------------------------------------------------------------------------+
|1  |Размер запроса в байтах|Отклонение от средних показателей 40-60 байт в сторону увеличения может говорить о скрытом канале|
|2  |Необычный тип трафика  |Если в трафике используются данные вида TXT, NULL, KEY - это говорит об аномалии                 |
|3  |Длина имени запроса    |Имена используемые злоумышленниками могут достигать 200 символов - средняя 30 символов           |
+---+-----------------------+-------------------------------------------------------------------------------------------------+



In [65]:
def ExtractData(files):
    data = []
    
    for file in files:
        
        for pkt in PcapReader(file):
            
            if pkt.haslayer(DNSQR):
                query = pkt.getlayer(DNSQR)
                
                qtype_field = query.get_field('qtype')
                qtype_data = qtype_field.i2repr(query, query.qtype)
                
                qname_field = query.get_field('qname')
                qname_data = qname_field.i2repr(query, query.qname)
                
                data.append((qtype_data, qname_data.__len__(), pkt.len))

    return data

In [66]:
def ConvertDataToRDD(data):
    return sparkContext.parallelize(data)

In [67]:
df_schema = StructType([StructField('TrafficType', StringType(), True),
                             StructField('NameLenght', IntegerType(), True),
                             StructField('Bytes', IntegerType(), True)])

In [69]:
data = ExtractData(files_list)

In [73]:
data.__len__()

86060

In [154]:
rdd = ConvertDataToRDD(data)
rdd

ParallelCollectionRDD[73] at readRDDFromFile at PythonRDD.scala:274

In [151]:
main_DF = spark.createDataFrame(rdd, df_schema)
main_DF

DataFrame[TrafficType: string, NameLenght: int, Bytes: int]

In [78]:
main_DF.write.parquet('C:\\Users\\igor.beryozkin\\Desktop\\docs\\traffic_data')

In [152]:
file_parquet = spark.read.parquet('C:\\Users\\igor.beryozkin\\Desktop\\docs\\traffic_data')

In [85]:
file_parquet.count(), file_parquet

(86060, DataFrame[TrafficType: string, NameLenght: int, Bytes: int])